In [11]:
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [12]:
s=pd.Series([1,2,3,np.nan,4,5])
s

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
5    5.0
dtype: float64

In [13]:
datas=pd.date_range('20160101',periods=6)
datas

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06'],
              dtype='datetime64[ns]', freq='D')

In [16]:
dataset = pd.read_csv('pm.csv')
dataset

,date,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,SE,1.79,0,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,SE,2.68,0,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,SE,3.57,0,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,SE,5.36,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [17]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [19]:
# load dataset
values = dataset.values
# integer encode direction
value

array([['2010-01-02 00:00:00', 129.0, -16, -4.0, 1020.0, 'SE', 1.79, 0,
        0],
       ['2010-01-02 01:00:00', 148.0, -15, -4.0, 1020.0, 'SE', 2.68, 0,
        0],
       ['2010-01-02 02:00:00', 159.0, -11, -5.0, 1021.0, 'SE', 3.57, 0,
        0],
       ['2010-01-02 03:00:00', 181.0, -7, -5.0, 1022.0, 'SE', 5.36, 1, 0],
       ['2010-01-02 04:00:00', 138.0, -7, -5.0, 1022.0, 'SE', 6.25, 2, 0]],
      dtype=object)

In [45]:
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
values[:,4]

array([0, 0, 1, 2, 2], dtype=object)

In [50]:
dataset = pd.read_csv('guifan.csv')
dataset

,var1(t-1) var2(t-1) var3(t-1) var4(t-1) var5(t-1) var6(t-1) var7(t-1) var8(t-1) var1(t)
0,0.129779 0.352941 0.245902 0.527273 0....
1,0.148893 0.367647 0.245902 0.527273 0....
2,0.159960 0.426471 0.229508 0.545454 0....
3,0.182093 0.485294 0.229508 0.563637 0....
4,0.138833 0.485294 0.229508 0.563637 0....


In [ ]:
trainx=[]

In [54]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        # the hidden_size is 51
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 1)

    def forward(self, input, future=0):
        outputs = []
        # both the input(h_t, c_t) and output(h_t2, c_t2) are initialized to zeros
        h_t = Variable(
            torch.zeros(input.size(0), 51), requires_grad=False)
        c_t = Variable(
            torch.zeros(input.size(0), 51), requires_grad=False)
        h_t2 = Variable(
            torch.zeros(input.size(0), 1), requires_grad=False)
        c_t2 = Variable(
            torch.zeros(input.size(0), 1), requires_grad=False)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(c_t, (h_t2, c_t2))
            outputs += [c_t2]

        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

NameError: name 'nn' is not defined

In [ ]:
# build the model
seq = Sequence()
criterion = nn.MSELoss()
# use LBFGS as optimizer since we can load the whole data to train
optimizer = optim.LBFGS(seq.parameters())

loss_list = []
test_loss_list = []
epoch_num = 50

# begin to train
for epoch in range(epoch_num):
    print('epoch : ', epoch)

    def closure():
        optimizer.zero_grad()
        out = seq(train_X)
        loss = criterion(out, train_y)
#         print('loss:', loss.data.numpy()[0])
        loss_list.append(loss.data.numpy()[0])
        loss.backward()
        return loss

    optimizer.step(closure)

    pred = seq(test_X)
    loss = criterion(pred, test_y)
#     print('test loss:', loss.data.numpy()[0])
    test_loss_list.append(loss.data.numpy()[0])

    y = pred.data.numpy()